<a href="https://colab.research.google.com/github/nasif-raihan/ML-and-DL-Codes/blob/main/GAN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Resorce](https://github.com/bnsreenu/python_for_microscopists/blob/master/125_126_GAN_training_mnist.py)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.layers import LeakyReLU

**Define input image dimensions
Large images take too much time and resources.**

In [2]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)

**Given input of noise (latent) vector, the Generator produces an image.**

**Alpha — α is a hyperparameter which controls the underlying value to which the
function saturates negatives network inputs.**

**Momentum — Speed up the training**

**Define your generator network.
Here we are only using Dense layers. But network can be complicated based on 
the application. For example, you can use VGG for super res. GAN.**

In [3]:
def build_generator():
  noise_shape = (100, )   # 1D array of size 100 (latent vector / noise)

  model = Sequential()
  
  model.add(Input(shape=(noise_shape)))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1025))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()

  noise = Input(shape=noise_shape)
  img = model(noise)  # Generated Image

  return Model(noise, img)

**Given an input image, the Discriminator outputs the likelihood of the image being real.**
  
**Binary classification - true or false (we're calling it validity)**

In [4]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  img = Input(shape=img_shape)
  validity = model(img)   #The validity is the Discriminator’s guess of input being real or not.

  return Model(img, validity)

**Load the Data:**

In [5]:
(X_train, _), (_, _) = mnist.load_data()
print(X_train.shape)

# Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

# Add channels dimension. As the input to our generator and discriminator has a shape 28x28x1.
X_train = np.expand_dims(X_train, axis=3)
X_train.shape

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)


(60000, 28, 28, 1)

**Now that we have constructed our two models it’s time to pit them against each other.**

**We do this by defining a training function, loading the data set, re-scaling our training images and setting the ground truths.**

***Epochs*** dictate the number of backward and forward propagations.

***Batch_size*** indicates the number of training samples per backward/forward propagation.

***sample_interval*** specifies after how many epochs we call our sample_image function.

In [6]:
def train(epochs, batch_size=128, save_interval=50):
  half_batch = int(batch_size / 2)
  """  
    We loop through a number of epochs to train our 
    Discriminator by first selecting a random batch of images from our true dataset, 
    generating a set of images from our Generator, 
    feeding both set of images into our Discriminator, and 
    finally setting the loss parameters for both the real and fake images, 
    as well as the combined loss.
  """ 

  for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half batch of real images
    # random.randint(low, high=None, size=None, dtype=int)
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    # random.normal(loc=0.0, scale=1.0, size=(int or tuple of ints, optional))  
    # Returns: ndarray or scalar
    noise = np.random.normal(0, 1, (half_batch, 100))

    # Generate a half batch of fake images
    gen_imgs = generator.predict(noise)

    """
      Train the discriminator on real and fake images, separately.
      Research showed that separate training is more effective.
    """

    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

    # Take average loss from real and fake images.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    """
      And within the same loop we train our Generator, by setting the input noise and
      ultimately training the Generator to have the Discriminator label its samples as valid
      by specifying the gradient loss.
    """
    # ---------------------
    #  Train Generator
    # ---------------------

    """
      Create noise vectors as input for generator. 
      Create as many noise vectors as defined by the batch size. 
      Based on normal distribution. Output will be of size (batch size, 100)
    """

    noise = np.random.normal(0, 1, (batch_size, 100))

    """
      The generator wants the discriminator to label the generated samples as valid (ones)
      This is where the genrator is trying to trick discriminator into believing the generated image is true (hence value of 1 for y)
    """

    valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size
    
    """
        Generator is a part of combined where it got directly linked with the discriminator
        Train the generator with noise as x and 1 as y. 
        Again, 1 as the output as it is adversarial and if generator did a great
        job of fooling the discriminator then the output would be 1 (true)
    """

    g_loss = combined.train_on_batch(noise, valid_y)

    """
      Additionally, in order for us to keep track of our training process, we print the
      progress and save the sample image output depending on the epoch interval specified.  
      Plot the progress.
    """
    
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    # If at save interval => save generated image samples
    if epoch % save_interval == 0:
        save_imgs(epoch)


**When the specific sample_interval is hit, we call the sample_image function. Which looks as follows:**

In [7]:
#This function saves our images for us to view

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

Let us also define our optimizer for easy use later on.
That way if you change your mind, you can change it easily here.

In [8]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

**Build and compile the discriminator first.**

**Generator will be trained as part of the combined model, later.**

**Pick the loss function and the type of metric to keep track.**                 

**Binary cross entropy as we are doing prediction and it is a better loss function compared to MSE or other.** 

Build and compile our Discriminator, pick the loss function.

In [9]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer, metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

Since we are only generating (faking) images, let us not track any metrics.

In [10]:
generator = build_generator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [11]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 512)              2048      
 hNormalization)                                      

In a GAN the Generator network takes noise z as an input to produce its images.

In [12]:
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

Now, in below, it ensures that when we combine our networks we only train the Generator.
While generator training we do not want discriminator weights to be adjusted. 
This Doesn't affect the above descriminator training. 

In [13]:
discriminator.trainable = False 

Below, it specifies that our Discriminator will take the images generated by our Generator and true dataset and set its output to a parameter called valid, which will indicate whether the input is real or not.

In [14]:
valid = discriminator(img)  #Validity check on the generated image

Here we combined the models and also set our loss function and optimizer. 
Again, we are only training the generator here. 
The ultimate goal here is for the Generator to fool the Discriminator.  
The combined model  (stacked generator and discriminator) takes:

`noise as input >> generates images >> determines validity`

In [15]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [16]:
!mkdir images

In [17]:
train(epochs=100, batch_size=32, save_interval=10)

0 [D loss: 0.562613, acc.: 50.00%] [G loss: 0.523691]
1 [D loss: 0.383469, acc.: 75.00%] [G loss: 0.552211]
2 [D loss: 0.370355, acc.: 65.62%] [G loss: 0.584345]
3 [D loss: 0.345811, acc.: 81.25%] [G loss: 0.726668]
4 [D loss: 0.302275, acc.: 93.75%] [G loss: 0.877413]
5 [D loss: 0.298396, acc.: 90.62%] [G loss: 1.047882]
6 [D loss: 0.251725, acc.: 100.00%] [G loss: 1.372830]
7 [D loss: 0.205758, acc.: 96.88%] [G loss: 1.498451]
8 [D loss: 0.145196, acc.: 100.00%] [G loss: 1.730555]
9 [D loss: 0.109174, acc.: 100.00%] [G loss: 1.810414]
10 [D loss: 0.115538, acc.: 100.00%] [G loss: 2.023898]
11 [D loss: 0.106259, acc.: 100.00%] [G loss: 2.088122]
12 [D loss: 0.091836, acc.: 100.00%] [G loss: 2.288846]
13 [D loss: 0.061167, acc.: 100.00%] [G loss: 2.499311]
14 [D loss: 0.072475, acc.: 100.00%] [G loss: 2.527823]
15 [D loss: 0.042798, acc.: 100.00%] [G loss: 2.706086]
16 [D loss: 0.047104, acc.: 100.00%] [G loss: 2.687873]
17 [D loss: 0.048316, acc.: 100.00%] [G loss: 2.835106]
18 [D los

Save model for future use to generate fake images.

In [18]:
generator.save('generator_model.h5')